# Understanding the Given Data


In [1]:
import pandas as pd
import numpy as np
import json


In [2]:
# Use the path to one of your CSV files
csv_path = r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Normal\Picking_up_objects\cam01_5m_picking_up_object_2023_2_16_17_22_17_000.csv'

# Read the CSV into a DataFrame
df = pd.read_csv(csv_path)

# Display the first 3 rows to see the structure
df.head(3)


,TIME,FRAMEID,PID,POSES,SCORES
0,2023-02-23 10:31:19.244127,1,0,"[334, 90, 334, 95, 321, 95, 318, 111, 320, 126...","[0.7808822393417358, 0.8417455554008484, 0.820..."
1,2023-02-23 10:31:19.309154,2,0,"[334, 90, 333, 94, 321, 94, 317, 111, 321, 126...","[0.7829259037971497, 0.8493701219558716, 0.823..."
2,2023-02-23 10:31:19.369036,3,0,"[334, 90, 333, 95, 321, 95, 318, 112, 321, 126...","[0.7636637091636658, 0.8489276170730591, 0.816..."



---

**Where are the X and Y coordinates?**

The X and Y coordinates are **flattened** within the `POSES` column string.

### Data Example

Consider the first line of your data:

```
POSES: "[525, 92, 532, 102, 527, 98, ..."
```

This string represents a single list of numbers that contains the coordinates of all 18 keypoints for one frame arranged sequentially.

### Coordinate Layout

Each frame consists of 36 numbers (18 pairs of X and Y coordinates) organized as follows:

- **Keypoint 0 (Nose):**
  - **X coordinate:** 525
  - **Y coordinate:** 92
- **Keypoint 1 (Neck):**
  - **X coordinate:** 532
  - **Y coordinate:** 102
- **...and so on for the remaining keypoints...**

### Keypoint Order

The order of these pairs corresponds to the keypoints labeled in the provided skeleton graph image:

- **Pair 0 (x0, y0):** Nose
- **Pair 1 (x1, y1):** Neck
- **Pair 2 (x2, y2):** Right Shoulder  
- **... and so on.**

### The SCORES Column

The `SCORES` column is structured in the same way. It is a flattened list of confidence scores corresponding to each keypoint:

- **score0:** Confidence for (x0, y0)
- **score1:** Confidence for (x1, y1)
- **... and so on.**

A score close to 0 indicates that the model was not confident about that keypoint's location, which may result in (x,y) pairs of (0,0).



In [3]:
# Get the 'POSES' string from the first row (index 0)
first_frame_poses_string = df['POSES'][0]

print("The data looks like this:")
print(first_frame_poses_string)
print("\nIts type is:")
print(type(first_frame_poses_string))

The data looks like this:
[334, 90, 334, 95, 321, 95, 318, 111, 320, 126, 346, 94, 350, 111, 350, 126, 325, 124, 328, 144, 329, 158, 343, 124, 342, 144, 342, 159, 334, 110, 0, 0, 0, 0, 334, 110]

Its type is:
<class 'str'>


In [4]:
# Use json.loads() to convert the string into a list of numbers
poses_list = json.loads(first_frame_poses_string)

print("The data now looks like this:")
print(poses_list)
print("\nIts type is:")
print(type(poses_list))
print(f"\nIt contains {len(poses_list)} numbers.")

The data now looks like this:
[334, 90, 334, 95, 321, 95, 318, 111, 320, 126, 346, 94, 350, 111, 350, 126, 325, 124, 328, 144, 329, 158, 343, 124, 342, 144, 342, 159, 334, 110, 0, 0, 0, 0, 334, 110]

Its type is:
<class 'list'>

It contains 36 numbers.


In [5]:
# Convert the Python list to a NumPy array
poses_array = np.array(poses_list)

# Reshape the array to have 18 rows and 2 columns (18 keypoints, 2 coords each)
structured_poses = poses_array.reshape(18, 2)

print("The final structured data for one frame:")
print(structured_poses)
print("\nThe shape of this array is:")
print(structured_poses.shape)

The final structured data for one frame:
[[334  90]
 [334  95]
 [321  95]
 [318 111]
 [320 126]
 [346  94]
 [350 111]
 [350 126]
 [325 124]
 [328 144]
 [329 158]
 [343 124]
 [342 144]
 [342 159]
 [334 110]
 [  0   0]
 [  0   0]
 [334 110]]

The shape of this array is:
(18, 2)


In [6]:
def parse_poses_from_string(poses_str: str) -> np.ndarray:
    """Takes the raw string from the 'POSES' column and returns a (18, 2) NumPy array."""
    pose_list = json.loads(poses_str)
    return np.array(pose_list).reshape(18, 2)

# Apply this function to every row in the 'POSES' column of our DataFrame
df['parsed_poses'] = df['POSES'].apply(parse_poses_from_string)

# Display the DataFrame again, now with our new structured column
df.head()

,TIME,FRAMEID,PID,POSES,SCORES,parsed_poses
0,2023-02-23 10:31:19.244127,1,0,"[334, 90, 334, 95, 321, 95, 318, 111, 320, 126...","[0.7808822393417358, 0.8417455554008484, 0.820...","[[334, 90], [334, 95], [321, 95], [318, 111], ..."
1,2023-02-23 10:31:19.309154,2,0,"[334, 90, 333, 94, 321, 94, 317, 111, 321, 126...","[0.7829259037971497, 0.8493701219558716, 0.823...","[[334, 90], [333, 94], [321, 94], [317, 111], ..."
2,2023-02-23 10:31:19.369036,3,0,"[334, 90, 333, 95, 321, 95, 318, 112, 321, 126...","[0.7636637091636658, 0.8489276170730591, 0.816...","[[334, 90], [333, 95], [321, 95], [318, 112], ..."
3,2023-02-23 10:31:19.423059,4,0,"[334, 90, 333, 95, 321, 95, 317, 112, 321, 126...","[0.7816575765609741, 0.8491430282592773, 0.818...","[[334, 90], [333, 95], [321, 95], [317, 112], ..."
4,2023-02-23 10:31:19.476460,5,0,"[334, 89, 333, 94, 321, 95, 317, 112, 320, 126...","[0.8119546175003052, 0.8538335561752319, 0.818...","[[334, 89], [333, 94], [321, 95], [317, 112], ..."


# Displaying a single CSV as an animation

In [7]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML # For displaying the animation in the notebook

In [8]:
# The connections define which keypoints to link with a line.
# This is based on the keypoint indices from your graph (0 to 17).
YOUR_SKELETON_CONNECTIONS = [
    [0, 1], [1, 2], [2, 3], [3, 4],    # Right arm
    [1, 5], [5, 6], [6, 7],            # Left arm
    [1, 14], [14, 8], [14, 11],       # Torso
    [8, 9], [9, 10],                   # Right leg
    [11, 12], [12, 13]                 # Left leg
]

# Get the list of (18, 2) arrays from the DataFrame column we created
pose_sequence = df['parsed_poses'].tolist()

In [ ]:
# --- Set up the plot ---
fig, ax = plt.subplots()

# --- Define the function that will update the plot for each frame ---
def update(frame_index):
    ax.clear() # Clear the previous frame's drawing

    keypoints = pose_sequence[frame_index]
    
    # Get all x and y coordinates to set the plot limits dynamically
    # We only consider points that were actually detected (not 0,0)
    x_coords = keypoints[:, 0][keypoints[:, 0] != 0]
    y_coords = keypoints[:, 1][keypoints[:, 1] != 0]

    if len(x_coords) > 0 and len(y_coords) > 0:
        # Plot the keypoints as dots
        ax.scatter(x_coords, y_coords, s=10, c='red') # s is size, c is color

        # Plot the limbs as lines
        for i, j in YOUR_SKELETON_CONNECTIONS:
            start_point = keypoints[i]
            end_point = keypoints[j]
            # Only draw if both points were detected
            if (start_point.any() and end_point.any()):
                 ax.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], 'b-')

        # Set the plot limits with a bit of padding
        ax.set_xlim(np.min(x_coords) - 20, np.max(x_coords) + 20)
        ax.set_ylim(np.min(y_coords) - 20, np.max(y_coords) + 20)
        ax.invert_yaxis() # Invert y-axis because image coordinates start from top-left

    ax.set_title(f"Frame {frame_index}")
    ax.set_aspect('equal', adjustable='box')


# --- Create the animation ---
# The 'frames' argument tells FuncAnimation how many times to call the update function
ani = FuncAnimation(fig, update, frames=len(pose_sequence), interval=100) # interval is ms between frames

# --- Display the animation in the notebook ---
# This might take a few moments to render
plt.close(fig) # Prevent a static image from displaying
HTML(ani.to_jshtml())

Animation size has reached 20979214 bytes, exceeding the limit of 20971520.0. If you're sure you want a larger animation embedded, set the animation.embed_limit rc parameter to a larger value (in MB). This and further frames will be dropped.


In [9]:
import pandas as pd
import numpy as np
import cv2
import json
import os

# --- Your final, polished visualization function ---
def save_sequence_as_video_final(pose_sequence: list, connections: list, output_path: str, fps: float = 15.0):
    """
    Takes a sequence of poses and saves it as a polished MP4 video file.
    FINAL VERSION: White background, smaller points, thinner lines.
    """
    if not pose_sequence:
        print(f"Warning: Empty pose sequence for {output_path}. Skipping.")
        return

    # Determine video dimensions dynamically
    all_x = [kp[0] for pose in pose_sequence for kp in pose if kp[0] != 0]
    all_y = [kp[1] for pose in pose_sequence for kp in pose if kp[1] != 0]
    
    if not all_x or not all_y:
        print(f"Warning: No valid keypoints in sequence for {output_path}. Skipping.")
        return
        
    width, height = int(np.max(all_x)) + 50, int(np.max(all_y)) + 50
    
    # Ensure output directory exists
    output_dir = os.path.dirname(output_path)
    if output_dir: os.makedirs(output_dir, exist_ok=True)

    # Setup video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    for keypoints in pose_sequence:
        frame = np.full((height, width, 3), 255, dtype=np.uint8)
        limb_thickness, point_radius = 2, 3
        
        for i, j in connections: # Draw limbs
            start_point, end_point = keypoints[i], keypoints[j]
            if start_point.any() and end_point.any():
                cv2.line(frame, (int(start_point[0]), int(start_point[1])), (int(end_point[0]), int(end_point[1])), (255, 0, 0), limb_thickness)

        for x, y in keypoints: # Draw points
            if x != 0 and y != 0:
                cv2.circle(frame, (int(x), int(y)), point_radius, (0, 0, 255), -1)
                
        video_writer.write(frame)
        
    video_writer.release()
    print(f"-> Saved: {output_path}")

def parse_poses_from_string(poses_str: str) -> np.ndarray:
    try:
        pose_list = json.loads(poses_str)
        return np.array(pose_list).reshape(18, 2)
    except (json.JSONDecodeError, ValueError):
        return np.zeros((18, 2))

# --- Main Script with CORRECTED File Paths ---

# Define the skeleton connections
YOUR_SKELETON_CONNECTIONS = [
    [0, 1], [1, 2], [2, 3], [3, 4], [1, 5], [5, 6], [6, 7],
    [1, 14], [14, 8], [14, 11], [8, 9], [9, 10], [11, 12], [12, 13]
]

# --- Corrected list of files to visualize ---
files_to_visualize = [
    # --- POTENTIAL SHOPLIFTER ---
    r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Potential_shoplifter\looking_right_left-hide_object\cam01_5m_state_looking_right_left-hide_object_2023_2_16_17_59_14_000.csv',
    r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Potential_shoplifter\looking_right_left-hide_object\cam02_3m_state_looking_right_left-hide_object_2023_2_16_17_59_14_000.csv',
    
    # --- NORMAL ---
    r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Normal\Picking_up_objects\cam01_5m_picking_up_object_2023_2_16_17_22_17_000.csv',
    r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Normal\Picking_up_objects\cam02_3m_picking_up_object_2023_2_16_17_22_17_001.csv',
    r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Normal\put_hand_in_pocket\cam01_5m_state_put_hand_in_pocket_2023_2_16_19_9_16_000.csv',
    r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Normal\put_hand_in_pocket\cam02_3m_state_put_hand_in_pocket_2023_2_16_19_9_16_000.csv',
    r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Normal\put_hand_in_pocket2\cam01_5m_state_put_hand_in_pocket_2023_2_16_18_46_39_000.csv',
    r'C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Normal\put_hand_in_pocket2\cam02_3m_state_put_hand_in_pocket_2023_2_16_18_46_39_000.csv'
]

# The base directory where all videos will be saved
base_output_dir = 'output_videos'

for csv_path in files_to_visualize:
    # A small adjustment to make the output path cleaner
    # It will remove the long prefix from your desktop
    path_parts = csv_path.split(os.sep)
    # We'll take the last 3 parts: Normal/Shoplifter -> Action -> Filename
    simple_path = os.path.join(*path_parts[-3:])
    output_video_path = os.path.join(base_output_dir, simple_path.replace('.csv', '.mp4'))
    
    # --- The rest of the loop is the same ---
    if not os.path.exists(csv_path):
        print(f"File not found: {csv_path}. Skipping.")
        continue
        
    print(f"Processing: {csv_path}")
    
    # Read and parse the data
    df = pd.read_csv(csv_path)
    pose_sequence = df['POSES'].apply(parse_poses_from_string).tolist()
    
    # Generate and save the video
    save_sequence_as_video_final(pose_sequence, YOUR_SKELETON_CONNECTIONS, output_video_path)

print("\n--- All selected videos have been generated! ---")

Processing: C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Potential_shoplifter\looking_right_left-hide_object\cam01_5m_state_looking_right_left-hide_object_2023_2_16_17_59_14_000.csv
-> Saved: output_videos\Potential_shoplifter\looking_right_left-hide_object\cam01_5m_state_looking_right_left-hide_object_2023_2_16_17_59_14_000.mp4
Processing: C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Potential_shoplifter\looking_right_left-hide_object\cam02_3m_state_looking_right_left-hide_object_2023_2_16_17_59_14_000.csv
-> Saved: output_videos\Potential_shoplifter\looking_right_left-hide_object\cam02_3m_state_looking_right_left-hide_object_2023_2_16_17_59_14_000.mp4
Processing: C:\Users\asus\Desktop\Ain-Guard Assesment\csvs_Skeleton_poses_normal_potential_shoplifter\Normal\Picking_up_objects\cam01_5m_picking_up_object_2023_2_16_17_22_17_000.csv
-> Saved: output_videos\Normal\Picking_up_objects\cam01_5m_picking